# N-glycosilation prediciton model

The model is a fine-tuned ESM2 model aimed to predict resiudes undergo N-glycosilation based on antigen protein sequence

The MIT License (MIT) Copyright (c) 2016 AYLIEN Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software. THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


## Requirements

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.environ['TORCH_HOME'] = "../torch_hub"

import pandas as pd
import numpy  as np

from copy import copy
from Bio import SeqIO

import torch
from torch.utils.data import Dataset
from torch import nn

import transformers
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import Trainer, TrainingArguments, EvalPrediction

import esm
from esm.pretrained import load_model_and_alphabet_hub

## Model and metrics

In [ ]:
class ESM2ForTokenClassification(nn.Module):

    def __init__(self, num_labels = 2):
        super().__init__()
        self.num_labels = num_labels    
        self.model_name = "esm2_t33_650M_UR50D"
        self.esm2, self.esm2_alphabet = load_model_and_alphabet_hub(self.model_name)         
        self.classifier = nn.Linear(1280, self.num_labels)

    def forward(self, token_ids):
        outputs = self.esm2.forward(token_ids, repr_layers=[33])['representations'][33]
        outputs = outputs[:,1:-1,:]
        logits = self.classifier(outputs)
        return SequenceClassifierOutput(logits=logits)
    
class MaskedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        print(inputs)
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = torch.nn.functional.cross_entropy(logits.view(-1, self.model.num_labels), labels.view(-1), 
                                                 ignore_index=-100))
        
        return (loss, outputs) if return_outputs else loss
    
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[:,:,1]
    batch_size, seq_len = preds.shape    
    pred_list, out_label_list = [], []

    for i in range(batch_size):
        for j in range(seq_len):
            if p.label_ids[i, j] != -100:
                out_label_list.append(p.label_ids[i][j])
                pred_list.append(preds[i][j])
    print(len(pred_list), len(out_label_list))
    return {
        "roc-auc": roc_auc_score(out_label_list, pred_list)
    }
    
def collator_fn(x):
    if len(x)==1:
        return x[0]
    print('x:', x)
    return x 

In [ ]:
class ESM2Dataset(Dataset):
    """
    A class to represent a sutable data set for model. 
    
    convert original pandas data frame to model set,
    where 'token ids' is ESM2 embedings corresponed to protein sequence 
    and 'lables' is a contact number values
    Attributes:
        df (pandas.DataFrame): dataframe with two columns: 
                0 -- preotein sequence in string ('GLVM') or list (['G', 'L', 'V', 'M']) format
                1 -- N-glycosilation label [0, 0, 1, -100, 0] format
        esm2_batch_converter (function):
                    ESM function callable to convert an unprocessed (labels + strings) batch to a
                    processed (labels + tensor) batch.
    """
    def __init__(self, df):
        self.df = df
        _, esm2_alphabet = esm.pretrained.esm2_t33_650M_UR50D()
        self.esm2_batch_converter = esm2_alphabet.get_batch_converter()

    def __getitem__(self, idx):
        item = {}
        _, _, esm2_batch_tokens = self.esm2_batch_converter([('' , ''.join(self.df.iloc[idx,0]))])
        item['labels'] = torch.unsqueeze(torch.LongTensor(self.df.iloc[idx, 1]), 0)
        item['token_ids'] = esm2_batch_tokens
        print(item)
        return item

    def __len__(self):
        return len(self.df)

# Data

### create train set from your Fasta file

In [ ]:
def loadTrainingSetFasta(fasta_file, onlyPositiveClass = False, ptm_aa={'N'}):
    fasta_sequences = SeqIO.parse(open(fasta_file), 'fasta')
    res = []
    for record in fasta_sequences:
        seq = np.array(record.seq)
        glyco_idx = [i for i, x in enumerate(seq) if x == '#']
        glyco_idx = [idx - n - 1 for n, idx in enumerate(glyco_idx)]
        n_glyco = len(glyco_idx)
        seq = seq[seq != '#']
        labels = np.array([0] * len(seq))
        labels[glyco_idx] = 1
        labels_balanc = copy(labels)
        idx = labels_balanc[labels_balanc == 0] ## select random len - n_glyco idx of 0 and set to -100
        if (set(seq[glyco_idx]) != ptm_aa) & (len(glyco_idx) != 0):
            print('ERROR', record.name)
            print(seq[glyco_idx])
        else:
            res.append([record.name, ''.join(seq), labels, sum(labels), len(labels)])
    return pd.DataFrame(res, columns= ['seq_id', 'seq', 'labels', 'n_glyco', 'seq_len'])
        

In [ ]:
def get_balance_lables(labels, coef=1):
    n_positive = len(np.where(labels==1)[0])
    n_negative = len(np.where(labels==0)[0])
    balance_labels = copy(labels)
    if n_negative > coef*n_positive:
        ids = np.where(labels==0)[0]
        random.shuffle(ids)
        ids = ids[: int((n_negative-coef*n_positive))]
        balance_labels[ids] = -100
    return balance_labels

In [ ]:
# df = loadTrainingSetFasta("XXX.fasta")
# df.loc[:,'balanced_labels'] = df.labels.apply(lambda x: get_balance_lables(x, 5))

### or upload example dataset

In [ ]:
train_df = pd.read_pickle('../data/Nglyco_train.pkl')
val_df = pd.read_pickle('../data/Nglyco_val.pkl')

In [ ]:
train_ds = ESM2Dataset(train_df.loc[['seq', 'balanced_labels']])
val_ds   = ESM2Dataset(val_df.loc[['seq', 'balanced_labels']])

## Model training

In [ ]:
training_args = TrainingArguments(
    output_dir='./test' ,          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=1,   # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    learning_rate=1e-05,             # learning rate
    weight_decay=0.0,                # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,               # How often to print logs
    save_strategy = "no",
    do_train=True,                   # Perform training
    do_eval=True,                    # Perform evaluation
    evaluation_strategy="epoch",     # evalute after each epoch
    gradient_accumulation_steps=4,  # total number of steps before back propagation
    fp16=False,                       # Use mixed precision
    run_name="N_binary",      # experiment name
    seed=42,                         # Seed for experiment reproducibility
    load_best_model_at_end=False,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,

)

In [ ]:
model = ESM2ForTokenClassification().cuda()

In [ ]:
trainer = MaskedTrainer(
    model= model,                # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset =train_ds,          # training dataset
    eval_dataset  =val_ds,             # evaluation dataset
    data_collator = collator_fn,
    compute_metrics = compute_metrics,    # evaluation metrics
)

In [ ]:
trainer.train()
#save weights
torch.save(trainer.model.state_dict(), 'esm2_glyco.pth')